# Exploring the 108 Runways with Complete Directional and Origin Location Information
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 23, 2021

***
These 108 records referencing Runways with complete directional and origin location information come from the 606 runway records within the 2158 unknown records of Standard Format Remarks.

## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

#### Import airports_runways_linked.csv dataset 

*Contains the cleaned and linked airports_cleaned.csv and rwy.csv supplemental datasets*

In [2]:
air_rwy = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')

In [3]:
air_rwy

,properties.GLOBAL_ID,properties.AIRPORT_ID,properties.DESIGNATOR,Runway.Latitude,Runway.Longitude,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,C747AD56-6CCF-4130-9255-643A4306CA51,656D38F0-F1FE-49A8-AB4F-677281616EF8,05/23,-176.645456,51.882484,ADK,Adak,51.883583,-176.642482,PADK
1,EA83C6B6-306D-4AD0-9BBC-3534B05CA636,F39AFCD2-D07F-4F41-96CF-08B79A271EAB,04/22,-154.181625,56.938950,AKK,Akhiok,56.938691,-154.182556,PAKH
2,6D8AC274-5B38-4994-8559-05B048032A21,C0EE48D3-E3AD-404E-945D-F404E345020D,01/19,-161.492556,60.914625,Z13,Akiachak,60.913810,-161.493329,PFZK
3,84E294FE-4120-472E-9186-7FE999B4BA7E,26D96486-FA29-4866-93EB-2EEEB7FA7144,E/W,-0.199980,-0.000081,KKI,Akiachak,60.907865,-161.435077,NaN
4,135362D2-1BD6-45A1-86B6-0B7BA2314384,26D96486-FA29-4866-93EB-2EEEB7FA7144,NW/SE,-0.199980,-0.000081,KKI,Akiachak,60.907865,-161.435077,NaN
...,...,...,...,...,...,...,...,...,...,...
24091,4F0E067C-87A7-4AFC-A820-A899EB4DCEC0,86E720F3-79ED-41F2-B8AD-7BAC41EE7F35,09/27,-87.100554,39.158473,NaN,NaN,NaN,NaN,NaN
24092,F5DDA7B5-E9F5-4432-9177-50E550AE39E2,A05B01BE-4052-4344-9EC0-937C3F78CF50,H1,-94.363796,45.967048,NaN,NaN,NaN,NaN,NaN
24093,4C580E1B-BB6C-422E-BCA9-45BD444E75A8,6B007B47-12DC-433A-907C-A17DF1F68704,H1,-93.709202,36.969451,NaN,NaN,NaN,NaN,NaN
24094,FB917C6C-DC51-4219-AB77-90D1624CB360,AF6FE437-1EEC-437C-9F10-CFE7B81CF4E8,H1,-94.322182,37.136954,NaN,NaN,NaN,NaN,NaN


#### Import other_rwy_totalinfo.csv dataset

*Contains the 108 records that reference Runways within the 2158 unknown records within the Standard Format. These 108 records contain both the origin location identifier and a distance (in NM).*

In [13]:
rwy_108 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/other_rwy_totalinfo.csv')

In [14]:
# Rename the 'UASLOCATION' field to 'IDENT' for merging purposes
rwy_108.rename(columns = {'UASLOCATION': 'IDENT'}, inplace = True)

In [15]:
rwy_108

,index,REMARKS,UAS_Location_Runways,IDENT,RWYLOCATION,UAS_Distance
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM']
1,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM']
2,110,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,RWY 36L,['6 NM']
3,125,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,RWY 16R,['4 NM']
4,146,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,RWY 15,['3 NM']
...,...,...,...,...,...,...
103,524,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,RWY 27,['2 NM']
104,596,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,RWY 25L,['3 NM']
105,599,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,RWY 25L,['10 NM']
106,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM']


#### Merge other_rwy_totalinfo.csv and airports_runways_linked.csv

*Merge the two datasets on the 'IDENT' field.*

In [16]:
rwy_linked = pd.merge(rwy_108, air_rwy, on = 'IDENT', how = 'left')

In [26]:
rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)

In [21]:
# Split 'RWYLOCATION' field into the RWY and its designator so we can link to the properties.DESIGNATOR field in order
# to obtain the correct latitude/longitude coordinates for the appropriate runway
rwydesignator = []

for i in range(len(rwy_linked)):
    rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
    rwydesignator.append(rwy_split[1])

In [28]:
# Add split column with RWY designators to dataframe as new field
rwy_linked['RWYDESIGNATOR'] = rwydesignator

In [29]:
rwy_linked

,index,REMARKS,UAS_Location_Runways,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,-85.570825,30.068523,Tyndall AFB,30.069047,-85.574981,KPAM,32R
1,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],EC5EA23A-FB43-4AF4-9F3A-6B72C1495F0C,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14R/32L,-85.579142,30.069627,Tyndall AFB,30.069047,-85.574981,KPAM,32R
2,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],D60C0EEA-7524-4790-A294-CE2112A0E2FB,B371E693-B91F-40ED-ABB1-628EFE9D3111,13L/31R,-97.013278,32.905492,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R
3,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],A4F0B104-48EA-445F-8DAB-850EE0FCBA8C,B371E693-B91F-40ED-ABB1-628EFE9D3111,13R/31L,-97.075234,32.901832,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R
4,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],A088B20F-FBE9-4C4F-9937-F7ABADEE30CC,B371E693-B91F-40ED-ABB1-628EFE9D3111,17C/35C,-97.026110,32.900981,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R
346,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],D1A26CC8-C4DA-48CF-A0B0-7FD883EBA2FA,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05R/23L,-78.790127,35.870421,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R
347,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],B270C60D-5C0C-41EA-8410-5DA7D520B4E3,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,14/32,-78.777558,35.870885,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R
348,604,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,RWY 10,['1 NM'],B2D0E0C7-E412-458E-B683-52C4DD4E4EA9,D339C530-1D36-4587-8130-9E9938BF474D,01/19,-81.200005,32.124275,Savannah/Hilton Head Intl,32.127583,-81.202139,KSAV,10


#### Check if RWY designator found in REMARK matches one of the RWY designators from the rwy.csv dataset

*If YES: RWYinREMARK field is 'True'*

*If NO: RWYinREMARK field is 'False'*
***
***NOTE:*** Since we linked on the 'IDENT' field, there are repeated remarks as a result of there being multiple runways at
each airport. Thus, we are only interested in the records for which the 'RWYinREMARK' field is 'True'.

In [31]:
rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

In [32]:
rwy_linked

,index,REMARKS,UAS_Location_Runways,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK
0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,-85.570825,30.068523,Tyndall AFB,30.069047,-85.574981,KPAM,32R,True
1,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],EC5EA23A-FB43-4AF4-9F3A-6B72C1495F0C,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14R/32L,-85.579142,30.069627,Tyndall AFB,30.069047,-85.574981,KPAM,32R,False
2,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],D60C0EEA-7524-4790-A294-CE2112A0E2FB,B371E693-B91F-40ED-ABB1-628EFE9D3111,13L/31R,-97.013278,32.905492,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,False
3,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],A4F0B104-48EA-445F-8DAB-850EE0FCBA8C,B371E693-B91F-40ED-ABB1-628EFE9D3111,13R/31L,-97.075234,32.901832,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,False
4,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],A088B20F-FBE9-4C4F-9937-F7ABADEE30CC,B371E693-B91F-40ED-ABB1-628EFE9D3111,17C/35C,-97.026110,32.900981,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,True
346,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],D1A26CC8-C4DA-48CF-A0B0-7FD883EBA2FA,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05R/23L,-78.790127,35.870421,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,False
347,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],B270C60D-5C0C-41EA-8410-5DA7D520B4E3,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,14/32,-78.777558,35.870885,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,False
348,604,Aircraft reported a fixed wing UAS pass within...,['RWY 10'],SAV,RWY 10,['1 NM'],B2D0E0C7-E412-458E-B683-52C4DD4E4EA9,D339C530-1D36-4587-8130-9E9938BF474D,01/19,-81.200005,32.124275,Savannah/Hilton Head Intl,32.127583,-81.202139,KSAV,10,False


In [35]:
other2158_runways98 = rwy_linked[rwy_linked['RWYinREMARK'] == True]

In [36]:
# Export final new_df to .csv 
other2158_runways98.to_csv('other2158_runways98.csv', index = False)

### Calculate Bearing Information for Runways

To do this, you take the number portion of the Runway Designator (in the 'RWYDESIGNATOR' field) and multiply it by 10 (effectively adding a 0 to the end)

***Example:*** *RWY 32R has designator 32R which indicates a bearing of 320 degrees*

In [42]:
other2158_runways98 = other2158_runways98.reset_index()

In [49]:
designator = other2158_runways98['RWYDESIGNATOR']
designator_bearing = []

for i in range(len(other2158_runways98)):
    designator_bearing.append(int(re.sub("\D","",designator[i]))*10)

In [51]:
other2158_runways98['RWY_BEARING'] = designator_bearing

In [52]:
other2158_runways98

,level_0,index,REMARKS,UAS_Location_Runways,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,Runway.Latitude,Runway.Longitude,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK,RWY_BEARING
0,0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,-85.570825,30.068523,Tyndall AFB,30.069047,-85.574981,KPAM,32R,True,320
1,8,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],ED9E6106-EB1D-401C-BCAE-E7A73A60A41C,B371E693-B91F-40ED-ABB1-628EFE9D3111,18R/36L,-97.054815,32.893723,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,True,180
2,12,110,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,RWY 36L,['6 NM'],DC9631DF-1821-449A-AD41-8EA0B8B40375,DA96A371-EB42-4312-8EAF-FA237DE1E404,18R/36L,-89.987278,35.039251,Memphis Intl,35.042411,-89.976679,KMEM,36L,True,360
3,15,125,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,RWY 16R,['4 NM'],D8DB5055-844E-44D9-A4DA-C7E399892BE1,5338FBAA-B183-4BC0-B3F1-27ABF6D1AD7C,16R/34L,-122.317932,47.449860,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,16R,True,160
4,16,146,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,RWY 15,['3 NM'],202E6282-A103-44B0-ACCE-3D984B66D091,3646423B-964B-466A-9CE9-3CB43457F772,15/33,-96.837304,32.970408,Addison,32.968556,-96.836444,KADS,15,True,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,335,524,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,RWY 27,['2 NM'],6AFB0072-D2C9-4832-9B3A-9BA008C1E9F9,265A7B00-46FF-4744-8673-499589FFA2BF,09/27,-84.346397,30.391355,Tallahassee Intl,30.396754,-84.350869,KTLH,27,True,270
94,340,596,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,RWY 25L,['3 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True,250
95,344,599,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,RWY 25L,['10 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,-118.404501,33.935085,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True,250
96,345,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,-78.792358,35.882177,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,True,230


### Convert Distances from NM to Kilometers

In [59]:
# Convert all distance from NM to kilometers
dist_kilo = []

for i in range(len(other2158_runways98)):
    distanceKilo = int(re.sub("\D","",other2158_runways98['UAS_Distance'][i]))* 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)
    
    

In [61]:
other2158_runways98['Distance_Kilometers'] = dist_kilo

# SWITCH Runway.Latitude and Runway.Longitude (I THINK ERIC HAS THESE MIXED UP ACCIDENTALLY)

In [65]:
other2158_runways98.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)

### Calculate UAS Latitude/Longitude Information using geopy library

In [67]:
uas_lat = []
uas_long = []


for i in range(len(other2158_runways98)):
    lat_rwy = pd.to_numeric(other2158_runways98['Runway_Latitude'][i])
    long_rwy = pd.to_numeric(other2158_runways98['Runway_Longitude'][i])
    b = pd.to_numeric(other2158_runways98['RWY_BEARING'][i])
    d = pd.to_numeric(other2158_runways98['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
other2158_runways98['UAS_Latitude'] = uas_lat
other2158_runways98['UAS_Longitude'] = uas_long

In [68]:
other2158_runways98

,level_0,index,REMARKS,UAS_Location_Runways,IDENT,RWYLOCATION,UAS_Distance,properties.GLOBAL_ID,properties.AIRPORT_ID,DESIGNATOR,...,NAME,LATITUDE,LONGITUDE,ICAO_ID,RWYDESIGNATOR,RWYinREMARK,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,0,4,"Aircraft observed a black UAS at 1,000 feet wh...",['RWY 32R'],PAM,RWY 32R,['3 NM'],8FAB3563-C164-4D2A-A7D7-4CEDC40E1B5D,9907F7DB-7C4F-4CB6-98FA-F0462EE27DE9,14L/32R,...,Tyndall AFB,30.069047,-85.574981,KPAM,32R,True,320,5.556,30.106912,-85.607878
1,8,86,Aircraft observed a UAS above while W bound at...,['RWY 18R'],DFW,RWY 18R,['5 NM'],ED9E6106-EB1D-401C-BCAE-E7A73A60A41C,B371E693-B91F-40ED-ABB1-628EFE9D3111,18R/36L,...,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,18R,True,180,9.260,32.810225,-97.054815
2,12,110,Aircraft observed a UAS at the same altitude w...,['RWY 36L'],MEM,RWY 36L,['6 NM'],DC9631DF-1821-449A-AD41-8EA0B8B40375,DA96A371-EB42-4312-8EAF-FA237DE1E404,18R/36L,...,Memphis Intl,35.042411,-89.976679,KMEM,36L,True,360,11.112,35.139411,-89.987278
3,15,125,Aircraft observed a red UAS off the right side...,['RWY 16R'],SEA,RWY 16R,['4 NM'],D8DB5055-844E-44D9-A4DA-C7E399892BE1,5338FBAA-B183-4BC0-B3F1-27ABF6D1AD7C,16R/34L,...,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,16R,True,160,7.408,47.387242,-122.284375
4,16,146,Aircraft observed a small red UAS E bound off ...,['RWY 15'],ADS,RWY 15,['3 NM'],202E6282-A103-44B0-ACCE-3D984B66D091,3646423B-964B-466A-9CE9-3CB43457F772,15/33,...,Addison,32.968556,-96.836444,KADS,15,True,150,5.556,32.927019,-96.807602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,335,524,Aircraft reported a UAS passing 100 feet below...,['RWY 27'],TLH,RWY 27,['2 NM'],6AFB0072-D2C9-4832-9B3A-9BA008C1E9F9,265A7B00-46FF-4744-8673-499589FFA2BF,09/27,...,Tallahassee Intl,30.396754,-84.350869,KTLH,27,True,270,3.704,30.391349,-84.384938
94,340,596,"Aircraft reported a UAS off the right side, ap...",['RWY 25L'],LAX,RWY 25L,['3 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,...,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True,250,5.556,33.917941,-118.460959
95,344,599,"Aircraft reported a grey, basketball size, UAS...",['RWY 25L'],LAX,RWY 25L,['10 NM'],8ED22315-4327-432F-9C74-65E1A06D97A5,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,07R/25L,...,Los Angeles Intl,33.942496,-118.408049,KLAX,25L,True,250,18.520,33.877836,-118.592608
96,345,602,Aircraft reported a UAS off the left side whil...,['RWY 23R'],RDU,RWY 23R,['3 NM'],91541445-E5CF-40C1-8E8F-3E759B9CA482,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,05L/23R,...,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,23R,True,230,5.556,35.849981,-78.839474


In [69]:
# Export final new_df to .csv 
other2158_runways98.to_csv('other2158_runways98_UAS_latlong.csv', index = False)